# Day 14 - Deepseek R1 Lite

In [1]:
# Read input from input.txt
with open('input.txt', 'r') as f:
    lines = f.readlines()

# Initialize grid with dimensions 103 (height) x 101 (width)
grid = [[0 for _ in range(101)] for _ in range(103)]

for line in lines:
    # Parse position and velocity
    parts = line.strip().split()
    p_part = parts[0].split('=')[1].split(',')
    v_part = parts[1].split('=')[1].split(',')
    x = int(p_part[0])
    y = int(p_part[1])
    vx = int(v_part[0])
    vy = int(v_part[1])
    
    # Calculate new position after 100 seconds with teleportation
    new_x = (x + vx * 100) % 101
    new_y = (y + vy * 100) % 103
    
    # Increment the count at the new position
    grid[new_y][new_x] += 1

# Define middle lines
middle_x = 50
middle_y = 51

# Initialize quadrant counts
q1 = 0
q2 = 0
q3 = 0
q4 = 0

# Sum counts in each quadrant, excluding middle lines
for y in range(103):
    for x in range(101):
        if x != middle_x and y != middle_y:
            if x > middle_x and y < middle_y:
                q1 += grid[y][x]
            elif x < middle_x and y < middle_y:
                q2 += grid[y][x]
            elif x < middle_x and y > middle_y:
                q3 += grid[y][x]
            elif x > middle_x and y > middle_y:
                q4 += grid[y][x]

# Calculate safety factor
safety_factor = q1 * q2 * q3 * q4

# Output the result
print(safety_factor)

229421808


## Part 2

In [2]:
import sys

def read_robots(file_path):
    robots = []
    with open(file_path, 'r') as f:
        for line in f:
            if not line.strip():
                continue
            parts = line.strip().split()
            p_part = parts[0].split('=')[1].split(',')
            v_part = parts[1].split('=')[1].split(',')
            x = int(p_part[0])
            y = int(p_part[1])
            vx = int(v_part[0])
            vy = int(v_part[1])
            robots.append((x, y, vx, vy))
    return robots

def simulate_positions(robots, t, width, height):
    positions = []
    for robot in robots:
        x = (robot[0] + robot[2] * t) % width
        y = (robot[1] + robot[3] * t) % height
        positions.append((x, y))
    return positions

def is_christmas_tree_pattern(positions, width, height):
    if not positions:
        return False
    # Find bounding box
    min_x = min(pos[0] for pos in positions)
    max_x = max(pos[0] for pos in positions)
    min_y = min(pos[1] for pos in positions)
    max_y = max(pos[1] for pos in positions)
    width_box = max_x - min_x + 1
    height_box = max_y - min_y + 1
    area_box = width_box * height_box
    num_robots = len(positions)
    # Heuristic: Check if the area is close to the number of robots and has a certain shape
    if abs(area_box - num_robots) > num_robots * 0.2:
        return False
    # Additional heuristic: Check aspect ratio for a tree-like shape (wider at the top)
    if width_box > height_box:
        return False
    # This is a simplistic check and may need refinement
    return True

def find_min_t_for_tree(robots, width, height, max_t=1000000):
    for t in range(max_t + 1):
        positions = simulate_positions(robots, t, width, height)
        if is_christmas_tree_pattern(positions, width, height):
            return t
    return -1  # Pattern not found within max_t seconds

def main():
    input_file = 'input.txt'
    width = 101
    height = 103
    robots = read_robots(input_file)
    max_t = 1000000  # Set a reasonable upper limit
    t = find_min_t_for_tree(robots, width, height, max_t)
    if t != -1:
        print(f"The fewest number of seconds required is: {t}")
    else:
        print("Pattern not found within the specified time limit.")

if __name__ == "__main__":
    main()

Pattern not found within the specified time limit.


In [3]:
def read_robots(file_path):
    robots = []
    with open(file_path, 'r') as f:
        for line in f:
            if not line.strip():
                continue
            parts = line.strip().split()
            p_part = parts[0].split('=')[1].split(',')
            v_part = parts[1].split('=')[1].split(',')
            x = int(p_part[0])
            y = int(p_part[1])
            vx = int(v_part[0])
            vy = int(v_part[1])
            robots.append((x, y, vx, vy))
    return robots

def simulate_positions(robots, t, width, height):
    positions = []
    for robot in robots:
        x = (robot[0] + robot[2] * t) % width
        y = (robot[1] + robot[3] * t) % height
        positions.append((x, y))
    return positions

def is_christmas_tree_pattern(positions, width, height, num_robots):
    row_counts = [0] * height
    for x, y in positions:
        row_counts[y] += 1
    
    # Find the row with the maximum count (likely the top of the tree)
    max_count_row = row_counts.index(max(row_counts))
    
    # Check if counts decrease as we move up from the max count row
    for y in range(max_count_row - 1):
        if row_counts[y] >= row_counts[y + 1]:
            return False
    
    # Check if counts decrease as we move down from the max count row
    for y in range(max_count_row, height - 1):
        if row_counts[y] <= row_counts[y + 1]:
            return False
    
    # Check for a dense cluster at the bottom for the trunk
    trunk_start = height - 5  # Assuming the trunk spans the last 5 rows
    trunk_robots = sum(row_counts[trunk_start:])
    if trunk_robots < num_robots * 0.2:
        return False
    
    return True

def find_min_t_for_tree(robots, width, height, num_robots, max_t=1000000):
    for t in range(max_t + 1):
        positions = simulate_positions(robots, t, width, height)
        if is_christmas_tree_pattern(positions, width, height, num_robots):
            return t
    return -1  # Pattern not found within max_t seconds

def main():
    input_file = 'input.txt'
    width = 101
    height = 103
    robots = read_robots(input_file)
    num_robots = len(robots)
    max_t = 1000000  # Set a reasonable upper limit
    t = find_min_t_for_tree(robots, width, height, num_robots, max_t)
    if t != -1:
        print(f"The fewest number of seconds required is: {t}")
    else:
        print("Pattern not found within the specified time limit.")

if __name__ == "__main__":
    main()

Pattern not found within the specified time limit.


In [4]:
def read_robots(file_path):
    robots = []
    with open(file_path, 'r') as f:
        for line in f:
            if not line.strip():
                continue
            parts = line.strip().split()
            p_part = parts[0].split('=')[1].split(',')
            v_part = parts[1].split('=')[1].split(',')
            x = int(p_part[0])
            y = int(p_part[1])
            vx = int(v_part[0])
            vy = int(v_part[1])
            robots.append((x, y, vx, vy))
    return robots

def simulate_positions(robots, t, width, height):
    positions = set()
    for robot in robots:
        x = (robot[0] + robot[2] * t) % width
        y = (robot[1] + robot[3] * t) % height
        positions.add((x, y))
    return positions

def is_christmas_tree_pattern(positions, width, height):
    # Define the expected Christmas tree pattern positions
    # For example, a simple tree with a few layers
    expected_positions = {
        (50, 0),  # Top of the tree
        (49, 1), (50, 1), (51, 1),
        (48, 2), (49, 2), (50, 2), (51, 2), (52, 2),
        (47, 3), (48, 3), (49, 3), (50, 3), (51, 3), (52, 3), (53, 3),
        (50, 4), (51, 4), (52, 4)  # Trunk of the tree
    }
    return positions == expected_positions

def find_min_t_for_tree(robots, width, height, max_t=10403):
    for t in range(max_t + 1):
        positions = simulate_positions(robots, t, width, height)
        if is_christmas_tree_pattern(positions, width, height):
            return t
    return -1  # Pattern not found within max_t seconds

def main():
    input_file = 'input.txt'
    width = 101
    height = 103
    robots = read_robots(input_file)
    max_t = 10403  # LCM of 101 and 103, since they are both prime
    t = find_min_t_for_tree(robots, width, height, max_t)
    if t != -1:
        print(f"The fewest number of seconds required is: {t}")
    else:
        print("Pattern not found within the specified time limit.")

if __name__ == "__main__":
    main()

Pattern not found within the specified time limit.


In [5]:
def read_robots(file_path):
    robots = []
    with open(file_path, 'r') as f:
        for line in f:
            if not line.strip():
                continue
            parts = line.strip().split()
            p_part = parts[0].split('=')[1].split(',')
            v_part = parts[1].split('=')[1].split(',')
            x = int(p_part[0])
            y = int(p_part[1])
            vx = int(v_part[0])
            vy = int(v_part[1])
            robots.append((x, y, vx, vy))
    return robots

def simulate_positions(robots, t, width, height):
    positions = []
    for robot in robots:
        x = (robot[0] + robot[2] * t) % width
        y = (robot[1] + robot[3] * t) % height
        positions.append((x, y))
    return positions

def is_christmas_tree_pattern(positions, height):
    row_counts = [0] * height
    for x, y in positions:
        row_counts[y] += 1
    
    # Find the index of the row with the maximum count
    max_count_index = row_counts.index(max(row_counts))
    
    # Check if counts increase up to max_count_index and then decrease
    # Allow some flexibility for the pattern
    # First, check increasing up to the peak
    for i in range(1, max_count_index):
        if row_counts[i] < row_counts[i - 1]:
            return False
    
    # Then, check decreasing after the peak
    for i in range(max_count_index + 1, height):
        if row_counts[i] > row_counts[i - 1]:
            return False
    
    return True

def find_min_t_for_tree(robots, width, height, max_t=1000):
    for t in range(max_t + 1):
        positions = simulate_positions(robots, t, width, height)
        if is_christmas_tree_pattern(positions, height):
            return t
    return -1  # Pattern not found within max_t seconds

def main():
    input_file = 'input.txt'
    width = 101
    height = 103
    robots = read_robots(input_file)
    max_t = 1000  # Set a reasonable upper limit
    t = find_min_t_for_tree(robots, width, height, max_t)
    if t != -1:
        print(f"The fewest number of seconds required is: {t}")
    else:
        print("Pattern not found within the specified time limit.")

if __name__ == "__main__":
    main()

Pattern not found within the specified time limit.


In [6]:
import numpy as np

def read_robots(file_path):
    robots = []
    with open(file_path, 'r') as f:
        for line in f:
            if not line.strip():
                continue
            parts = line.strip().split()
            p_part = parts[0].split('=')[1].split(',')
            v_part = parts[1].split('=')[1].split(',')
            x = int(p_part[0])
            y = int(p_part[1])
            vx = int(v_part[0])
            vy = int(v_part[1])
            robots.append((x, y, vx, vy))
    return np.array(robots)

def simulate_positions(robots, t, width, height):
    x = (robots[:,0] + robots[:,2] * t) % width
    y = (robots[:,1] + robots[:,3] * t) % height
    return np.column_stack((x, y))

def is_christmas_tree_pattern(row_counts, height):
    mode = np.argmax(row_counts)
    # Check increasing up to mode
    for i in range(1, mode):
        if row_counts[i] < row_counts[i - 1]:
            return False
    # Check decreasing after mode
    for i in range(mode + 1, height):
        if row_counts[i] > row_counts[i - 1]:
            return False
    return True

def find_min_t_for_tree(robots, width, height, max_t=100000):
    for t in range(max_t + 1):
        positions = simulate_positions(robots, t, width, height)
        row_counts = np.bincount(positions[:,1], minlength=height)
        if is_christmas_tree_pattern(row_counts, height):
            return t
    return -1

def main():
    input_file = 'input.txt'
    width = 101
    height = 103
    robots = read_robots(input_file)
    max_t = 100000  # Set a reasonable upper limit
    t = find_min_t_for_tree(robots, width, height, max_t)
    if t != -1:
        print(f"The fewest number of seconds required is: {t}")
    else:
        print("Pattern not found within the specified time limit.")

if __name__ == "__main__":
    main()

Pattern not found within the specified time limit.
